In [1]:
import csv
import os
import time

In [2]:
def read_csv(path):
    """
    Reads the CSV file at path, and returns a list of rows. Each row is a
    dictionary that maps a column name to a value in that column, as a string.
    """
    output = []
    for row in csv.DictReader(open(path)):
        output.append(row)
    return output


In [3]:
def row_to_edge(row):
    """
    Given an election result row or poll data row, returns the Democratic edge
    in that state.
    """
    return float(row["Dem"]) - float(row["Rep"])  

def state_edges(election_result_rows):
    """
    Given a list of election result rows, returns state edges.
    The input list does has no duplicate states;
    that is, each state is represented at most once in the input list.
    """
    dic={}
    for i in election_result_rows:
        dic.update({ i["State"]: row_to_edge(i)})
    return dic

In [4]:
def earlier_date(date1, date2):
    """
    Given two dates as strings (formatted like "Oct 06 2012"), returns True if 
    date1 is after date2.
    """
    return (time.strptime(date1, "%b %d %Y") < time.strptime(date2, "%b %d %Y"))

def most_recent_poll_row(poll_rows, pollster, state):
    """
    Given a list of poll data rows, returns the most recent row with the
    specified pollster and state. If no such row exists, returns None.
    """
    recent_date = "Oct 01 1500"
    data = None
    for i in poll_rows:
        if i["State"] == state and i["Pollster"] == pollster:
            if earlier_date(recent_date, i["Date"]):
                recent_date = i["Date"]
                data = i
    return data


In [5]:
def unique_column_values(rows, column_name):
    """
    Given a list of rows and the name of a column (a string), returns a set
    containing all values in that column.
    """
    my_set = set()
    for i in rows:
        if i[column_name] not in my_set:
            my_set.add(i[column_name])
    return my_set

def pollster_predictions(poll_rows):
    """
    Given a list of poll data rows, returns pollster predictions.
    """
    dic = {}
    Pollster_set = unique_column_values(poll_rows, "Pollster")
    State_set = unique_column_values(poll_rows, "State")
    for i in Pollster_set:
        temp = []
        for j in State_set:
            a = most_recent_poll_row(poll_rows, i, j)
            if a is not None:
                temp.append(a)
                b = state_edges(temp)
                dic[i] = b            
    return dic

In [6]:
def average_error(state_edges_predicted, state_edges_actual):
    """
    Given predicted state edges and actual state edges, returns
    the average error of the prediction.
    """
    error = 0
    count = 0
    predict_set = set(state_edges_predicted)
    actual_set = set(state_edges_actual)
    for name in predict_set.intersection(actual_set):
        error += abs(state_edges_predicted[name] - state_edges_actual[name])
        count += 1
    return (error/count)

def pollster_errors(pollster_predictions, state_edges_actual):
    """
    Given pollster predictions and actual state edges, retuns pollster errors.
    """
    dic = {}
    for innerDict in pollster_predictions:                        ###innerDict -> Pollster
        Pollster_predict = pollster_predictions[innerDict]        ###{'WA': 1.0, 'CA': -2.0, 'ID': -20.0}
        avg_err = average_error(Pollster_predict, state_edges_actual)
        dic.update({innerDict: avg_err})
    return dic

In [7]:
def pivot_nested_dict(nested_dict):
    """
    Pivots a nested dictionary, producing a different nested dictionary
    containing the same values.
    The input is a dictionary d1 that maps from keys k1 to dictionaries d2,
    where d2 maps from keys k2 to values v.
    The output is a dictionary d3 that maps from keys k2 to dictionaries d4,
    where d4 maps from keys k1 to values v.
    For example:
      input = { "a" : { "x": 1, "y": 2 },s
                "b" : { "x": 3, "z": 4 } }
      output = {'y': {'a': 2},
                'x': {'a': 1, 'b': 3},
                'z': {'b': 4} }
    """

    from collections import defaultdict
    out = defaultdict(dict)
    for outer_key, inner in nested_dict.items():
        for inner_key, value in inner.items():
            out[inner_key][outer_key] = value
    return out

In [8]:
def average_error_to_weight(error):
    """
    Given the average error of a pollster, returns that pollster's weight.
    The error must be a positive number.
    """
    return error ** (-2)

# The default average error of a pollster who did no polling in the
# previous election.
DEFAULT_AVERAGE_ERROR = 5.0


In [9]:
def pollster_to_weight(pollster, pollster_errors):
    """"
    Given a pollster and a pollster errors, return the given pollster's weight.
    """
    if pollster not in pollster_errors:
        weight = average_error_to_weight(DEFAULT_AVERAGE_ERROR)
    else:
        weight = average_error_to_weight(pollster_errors[pollster])
    return weight

In [10]:
def weighted_average(items, weights):
    """
    Returns the weighted average of a list of items.
    
    Arguments:
    items is a list of numbers.
    weights is a list of numbers, whose sum is nonzero.
    
    Each weight in weights corresponds to the item in items at the same index.
    items and weights must be the same length.
    """
    assert len(items) > 0
    assert len(items) == len(weights)
    total = 0
    sum_weights = 0
    for i in range(len(items)):
        total += items[i] * weights[i]
        sum_weights += weights[i]
    return total/sum_weights


In [11]:
def average_edge(pollster_edges, pollster_errors):
    """
    Given pollster edges and pollster errors, returns the average of these edges
    weighted by their respective pollster errors.
    """
    from decimal import Decimal
    weight_sum = 0
    denum = 0
    weight = 0
    for i in pollster_edges:
        weight = Decimal(pollster_to_weight(i, pollster_errors))
        weight_sum += Decimal(weight)
        denum += Decimal(weight)*Decimal(pollster_edges[i]) 
    result = float(denum)/float(weight_sum)
    return result

In [12]:
def predict_state_edges(pollster_predictions, pollster_errors):
    """
    Given pollster predictions from a current election and pollster errors from
    a past election, returns the predicted state edges of the current election.
    """
    dic = {}
    dic_trans = pivot_nested_dict(pollster_predictions)
    for i in dic_trans:
        edges = dic_trans[i]
        result = average_edge(edges, pollster_errors)
        dic.update({ i: result})
    return dic

In [13]:
def electoral_college_outcome(ec_rows, state_edges):
    """
    Given electoral college rows and state edges, returns the outcome of
    the Electoral College, as a map from "Dem" or "Rep" to a number of
    electoral votes won.  If a state has an edge of exactly 0.0, its votes
    are evenly divided between both parties.
    """
    ec_votes = {}               # maps from state to number of electoral votes
    for row in ec_rows:
        ec_votes[row["State"]] = float(row["Electors"])

    outcome = {"Dem": 0, "Rep": 0}
    for state in state_edges:
        votes = ec_votes[state]
        if state_edges[state] > 0:
            outcome["Dem"] += votes
        elif state_edges[state] < 0:
            outcome["Rep"] += votes
        else:
            outcome["Dem"] += votes/2.0
            outcome["Rep"] += votes/2.0
    return outcome


In [14]:
def print_dict(dictionary):
    """
    Given a dictionary, prints its contents in sorted order by key.
    Rounds float values to 8 decimal places.
    """
    for key in sorted(dictionary.keys()):
        value = dictionary[key]
        if type(value) == float:
            value = round(value, 8)
        print(key, value)


In [15]:
def main():
    """
    Main function, which is executed when election.py is run as a Python script.
    """
    # Read state edges from the 2008 election
    edges_2008 = state_edges(read_csv("data/2008-results.csv"))
    
    # Read pollster predictions from the 2008 and 2012 election
    polls_2008 = pollster_predictions(read_csv("data/2008-polls.csv"))
    polls_2012 = pollster_predictions(read_csv("data/2012-polls.csv"))
    
    # Compute pollster errors for the 2008 election
    error_2008 = pollster_errors(polls_2008, edges_2008)
    
    # Predict the 2012 state edges
    prediction_2012 = predict_state_edges(polls_2012, error_2008)
    
    # Obtain the 2012 Electoral College outcome
    ec_2012 = electoral_college_outcome(read_csv("data/2012-electoral-college.csv"),
                                        prediction_2012)
    
    print("Predicted 2012 election results:")
    print_dict(prediction_2012)
    print
    
    print("Predicted 2012 Electoral College outcome:")
    print_dict(ec_2012)
    print    


# If this file, election.py, is run as a Python script (such as by typing
# "python election.py" at the command shell), then run the main() function.
if __name__ == "__main__":
    main()

Predicted 2012 election results:
AK -22.0
AL -18.0
AR -24.122398
AZ -4.66430118
CA 14.11583444
CO 1.20293566
CT 12.84410977
DC 80.0
DE 25.0
FL 0.57143612
GA -9.30541301
HI 27.0
IA 2.15816992
ID -36.0
IL 19.04412837
IN -9.47132082
KS -20.0
KY -14.0
LA -11.63974839
MA 20.61154303
MD 23.44739426
ME 13.07697933
MI 6.21542005
MN 7.88152447
MO -9.20776204
MS -6.0
MT -8.60944804
NC -1.08747954
ND -17.24462124
NE -14.70698124
NH 1.49398848
NJ 12.1577076
NM 9.90780976
NV 2.949298
NY 26.72006693
OH 3.82265211
OK -25.0
OR 6.08883386
PA 4.47316823
RI 19.37311842
SC -6.0
SD -9.0
TN -9.86945113
TX -15.81828931
UT -45.24755722
VA 3.45068238
VT 37.0
WA 15.28608023
WI 7.49682045
WV -14.0
WY -32.0
Predicted 2012 Electoral College outcome:
Dem 332.0
Rep 206.0
